In [ ]:
#! pip install selenium
!pip install chromedriver-autoinstaller
!pip install webdriver-manager

In [ ]:
import requests
from requests import get
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from readability import Readability
import csv
from datetime import datetime as dt
import time

# open file with hotel data inside.
hotel_data = open('Hotel_Data.csv', 'a', newline='', encoding='utf-8')
csvWriter = csv.writer(hotel_data)
# write headers if necessary
#csvWriter.writerow(["Hotel", "Titles", "Reviews", "Dates", "Review Rating", "Travel Type"])

# opening file with hotel urls
with open('hotels_notaccor.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
# function for scraping reviews from hotels with over 1000 reviews.
def hotel_scrape(url_here):
    # initiate driver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    # Make a request to the website and get the object
    driver.get(url_here)
    # grab the hotel name
    Hotel = driver.find_element_by_xpath(".//h1[@class='QdLfr b d Pn']").get_attribute("innerHTML")

    # for each page loop
    i = 0
    for i in range(100):
        time.sleep(1)

        # expand the review
        expander = WebDriverWait(driver, 120).until(
            EC.element_to_be_clickable(
                (By.XPATH, ".//span[(@class='Ignyf _S Z')]")))
        expander.click()

        # grab the number of reviews displayed on page
#         time.sleep(3)
        #container = driver.find_elements_by_xpath("//div[@data-reviewid]")
        container = driver.find_elements_by_xpath("//div[@class='WAllg _T']")

        x = 0
        for x in range(len(container)):

            # grab the title
            try:
                title = container[x].find_element_by_xpath(".//div[contains(@data-test-target, 'review-title')]").text
            except (NoSuchElementException) as err:
                title = "NA"

            # grab the review
            try:
                review = container[x].find_element_by_xpath(".//span[@class='QewHA H4 _a']").text
            except (NoSuchElementException) as err:
                review = "NA"

            # grab the date
            try:
                date = container[x].find_element_by_xpath(".//span[@class='teHYY _R Me S4 H3']").text.partition(":")[2]
            except (NoSuchElementException) as err:
                date = "NA"

            # grab the rating
            try:
                rating = container[x].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
                rating = (int(rating))/10
            except (NoSuchElementException) as err:
                rating = "NA"

            # grab the traveller type
            try:
                travelType = container[x].find_element_by_xpath(".//span[@class='TDKzw _R Me']").get_attribute("innerHTML")
                travelType = travelType.partition("n>")[2]
            except (NoSuchElementException) as err:
                travelType = "NA"

            # write review to csv
            csvWriter.writerow([Hotel, title, review, date, rating, travelType])

        #print(x, i)

        # click next page once last review is processed
        driver.find_element_by_xpath('.//a[@class="ui_button nav next primary "]').click()

    driver.quit()

In [ ]:
# function for scraping hotels with less than 1000 reviews
def hotel_scrape2(url_here):
    # initiate driver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    # Make a request to the website and get the object
    driver.get(url_here)
    # grab the hotel name
    Hotel = driver.find_element_by_xpath(".//h1[@class='QdLfr b d Pn']").get_attribute("innerHTML")
    reviewCnt = driver.find_elements_by_xpath("//span[@class='POjZy']")
    reviewCnt2 = reviewCnt[1].get_attribute("innerHTML").partition(">")[2].partition("<")[0]
    print(reviewCnt2)
    reviewCnt3 = ((int(reviewCnt2))/10)-1
    reviewCnt3 = math.floor(reviewCnt3)
    # for each page loop
    i = 0
    for i in range(reviewCnt3):
        time.sleep(1)

        # expand the review
        expander = WebDriverWait(driver, 120).until(
            EC.element_to_be_clickable(
                (By.XPATH, ".//span[(@class='Ignyf _S Z')]")))
        expander.click()

        # grab the number of reviews displayed on page
#         time.sleep(3)
        #container = driver.find_elements_by_xpath("//div[@data-reviewid]")
        container = driver.find_elements_by_xpath("//div[@class='WAllg _T']")

        x = 0
        for x in range(len(container)):

            # grab the title
            try:
                title = container[x].find_element_by_xpath(".//div[contains(@data-test-target, 'review-title')]").text
            except (NoSuchElementException) as err:
                title = "NA"

            # grab the review
            try:
                review = container[x].find_element_by_xpath(".//span[@class='QewHA H4 _a']").text
            except (NoSuchElementException) as err:
                review = "NA"

            # grab the date
            try:
                date = container[x].find_element_by_xpath(".//span[@class='teHYY _R Me S4 H3']").text.partition(":")[2]
            except (NoSuchElementException) as err:
                date = "NA"

            # grab the rating
            try:
                rating = container[x].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
                rating = (int(rating))/10
            except (NoSuchElementException) as err:
                rating = "NA"

            # grab the traveller type
            try:
                travelType = container[x].find_element_by_xpath(".//span[@class='TDKzw _R Me']").get_attribute("innerHTML")
                travelType = travelType.partition("n>")[2]
            except (NoSuchElementException) as err:
                travelType = "NA"

            # write review to csv
            csvWriter.writerow([Hotel, title, review, date, rating, travelType])

        #print(x, i)

        # click next page once last review is processed
        driver.find_element_by_xpath('.//a[@class="ui_button nav next primary "]').click()

    driver.quit()

In [ ]:
# executing scraper function. if hotel has more than 1000 reviews, use hotel_scrape function
for link in range(len(data)):
    url = data[link][1]
    hotel = data[link][0]
    # start time
    startTime = dt.now()
    hotel_scrape2(url)
    # finish time
    endTime = dt.now()
    timeTaken = endTime - startTime
    print(timeTaken, hotel)
    time.sleep(3)

283
0:01:20.708237 Wanderlust
220
0:01:06.602156 Aqueen Heritage Hotel Joo Chiat
197
0:00:55.346751 Hilton Singapore Orchard
203
0:01:03.705860 Vibe Hotel Singapore Orchard
143
0:00:46.155171 Ann Siang House
153
0:00:51.177735 KeSa House
